In [3]:
import numpy as np
from algorithms.greedy_mips import generate_conditer, greedy_mips  

In [4]:
# Generate synthetic test data
def generate_test_data(num_atoms, len_signal):
    """
    Generate random test data for atoms and signal vectors.
    :param num_atoms: Number of atoms (data points)
    :param len_signal: Length of each vector
    :return: Tuple of atoms and signals
    """
    np.random.seed(42)  # Set random seed for reproducibility
    atoms = np.random.randn(num_atoms, len_signal)
    signals = np.random.randn(1, len_signal)  # Single signal
    return atoms, signals

# Parameters for testing
num_atoms = 100  # Number of atoms
len_signal = 50  # Length of each vector
atoms, signals = generate_test_data(num_atoms, len_signal)

# Generate conditer for query-independent preprocessing
conditer = generate_conditer(atoms)

# Run Greedy MIPS
budget = 10  # Number of atoms for naive computation
num_best_atoms = 5  # Number of top atoms to retrieve
best_atoms, query_times, ranking_times, complexities = greedy_mips(
    atoms=atoms,
    signals=signals,
    budget=budget,
    num_best_atoms=num_best_atoms,
    conditer=conditer,
    verbose=True
)

# Output results
print(f"Best {num_best_atoms} atoms (indices) for each signal:\n{best_atoms}")
print(f"Query-dependent runtime (seconds):\n{query_times}")
print(f"Candidate ranking runtime (seconds):\n{ranking_times}")
print(f"Total complexities:\n{complexities}")

# Naive approach for validation
inner_products = np.dot(atoms, signals[0])
top_k_naive = np.argsort(inner_products)[-num_best_atoms:][::-1]
print(f"Top {num_best_atoms} candidates using naive approach (indices): {top_k_naive}")

# Compare accuracy
accuracy = len(np.intersect1d(best_atoms[0], top_k_naive)) / num_best_atoms
print(f"Accuracy compared to naive method: {accuracy * 100:.2f}%")

# Compute speedup
total_naive_computations = num_atoms * len_signal * len(signals)
average_complexity = complexities.mean()
speedup_ratio = total_naive_computations / average_complexity
print(f"Speedup ratio: {speedup_ratio:.2f} times faster than naive computation.")


preprocess time: 0.0
query dependent preprocess + candidates screening: 3.3665082454681396
candidate ranking: 0.0
Best 5 atoms (indices) for each signal:
[[31  7 26 73 12]]
Query-dependent runtime (seconds):
[3.36650825]
Candidate ranking runtime (seconds):
[0.]
Total complexities:
[3430]
Top 5 candidates using naive approach (indices): [21 33 27 38 45]
Accuracy compared to naive method: 0.00%
Speedup ratio: 1.46 times faster than naive computation.


Movie Lens 100k

In [5]:
import numpy as np
from sklearn.decomposition import TruncatedSVD
from surprise import Dataset, Reader
from surprise.model_selection import train_test_split


# Load the MovieLens dataset (100k version for simplicity)
reader = Reader(line_format='user item rating timestamp', sep='\t')
data = Dataset.load_builtin('ml-100k')
trainset, testset = train_test_split(data, test_size=0.2)

# Build a user-item matrix
user_item_matrix = np.zeros((trainset.n_users, trainset.n_items))
for uid, iid, rating in trainset.all_ratings():
    user_item_matrix[int(uid), int(iid)] = rating

# Use SVD to reduce dimensionality
svd = TruncatedSVD(n_components=50)
atoms = svd.fit_transform(user_item_matrix.T)  # Transpose to get item embeddings

# Create a signal (simulate a user's preference vector)
user_id = 0  # Choose a user ID from the dataset
user_ratings = user_item_matrix[user_id]
signal = np.dot(user_ratings, atoms)  # Weighted average of rated item embeddings
signals = signal[np.newaxis, :]  # Ensure signal has 2D shape for batch processing

# Initialize Greedy MIPS parameters
budget = 10  # Number of candidates to screen
num_best_atoms = 10  # Top-k items to retrieve
conditer = generate_conditer(atoms)  # Query-independent preprocessing

# Run Greedy MIPS
best_atoms, query_times, ranking_times, complexities = greedy_mips(
    atoms=atoms,
    signals=signals,
    budget=budget,
    num_best_atoms=num_best_atoms,
    conditer=conditer,
    verbose=True
)

# Output results
print(f"Top {num_best_atoms} candidates (indices):\n{best_atoms[0]}")
print(f"Query-dependent runtime (seconds): {query_times[0]:.4f}")
print(f"Candidate ranking runtime (seconds): {ranking_times[0]:.4f}")
print(f"Total complexity: {complexities[0]}")

# Naive approach for validation
inner_products = np.dot(atoms, signal)
top_k_naive = np.argsort(inner_products)[-num_best_atoms:][::-1]
print(f"Top {num_best_atoms} candidates using naive approach (indices): {top_k_naive}")

# Compare results
accuracy = len(np.intersect1d(best_atoms[0], top_k_naive)) / num_best_atoms
print(f"Accuracy compared to naive method: {accuracy * 100:.2f}%")

# Compute speedup ratio
total_naive_computations = atoms.shape[0] * atoms.shape[1]
average_complexity = complexities.mean()
speedup_ratio = total_naive_computations / average_complexity
print(f"Speedup ratio: {speedup_ratio:.2f} times faster than naive computation.")


preprocess time: 0.0
query dependent preprocess + candidates screening: 0.0009822845458984375
candidate ranking: 0.0
Top 10 candidates (indices):
[ 20  33 225  22  66 287 101 185 169 414]
Query-dependent runtime (seconds): 0.0010
Candidate ranking runtime (seconds): 0.0000
Total complexity: 3430
Top 10 candidates using naive approach (indices): [ 20  33 225  22  66  73 287 101 351 112]
Accuracy compared to naive method: 70.00%
Speedup ratio: 24.10 times faster than naive computation.


Movie Lens 1m

In [6]:
import numpy as np
from sklearn.decomposition import TruncatedSVD
from surprise import Dataset, Reader
from surprise.model_selection import train_test_split

# Load the MovieLens dataset (100k version for simplicity)
reader = Reader(line_format='user item rating timestamp', sep='\t')
data = Dataset.load_builtin('ml-1m')
trainset, testset = train_test_split(data, test_size=0.2)

# Build a user-item matrix
user_item_matrix = np.zeros((trainset.n_users, trainset.n_items))
for uid, iid, rating in trainset.all_ratings():
    user_item_matrix[int(uid), int(iid)] = rating

# Use SVD to reduce dimensionality
svd = TruncatedSVD(n_components=50)
atoms = svd.fit_transform(user_item_matrix.T)  # Transpose to get item embeddings

# Create a signal (simulate a user's preference vector)
user_id = 0  # Choose a user ID from the dataset
user_ratings = user_item_matrix[user_id]
signal = np.dot(user_ratings, atoms)  # Weighted average of rated item embeddings
signals = signal[np.newaxis, :]  # Ensure signal has 2D shape for batch processing

# Initialize Greedy MIPS parameters
budget = 10  # Number of candidates to screen
num_best_atoms = 10  # Top-k items to retrieve
conditer = generate_conditer(atoms)  # Query-independent preprocessing

# Run Greedy MIPS
best_atoms, query_times, ranking_times, complexities = greedy_mips(
    atoms=atoms,
    signals=signals,
    budget=budget,
    num_best_atoms=num_best_atoms,
    conditer=conditer,
    verbose=True
)

# Output results
print(f"Top {num_best_atoms} candidates (indices):\n{best_atoms[0]}")
print(f"Query-dependent runtime (seconds): {query_times[0]:.4f}")
print(f"Candidate ranking runtime (seconds): {ranking_times[0]:.4f}")
print(f"Total complexity: {complexities[0]}")

# Naive approach for validation
inner_products = np.dot(atoms, signal)
top_k_naive = np.argsort(inner_products)[-num_best_atoms:][::-1]
print(f"Top {num_best_atoms} candidates using naive approach (indices): {top_k_naive}")

# Compare results
accuracy = len(np.intersect1d(best_atoms[0], top_k_naive)) / num_best_atoms
print(f"Accuracy compared to naive method: {accuracy * 100:.2f}%")

# Compute speedup ratio
total_naive_computations = atoms.shape[0] * atoms.shape[1]
average_complexity = complexities.mean()
speedup_ratio = total_naive_computations / average_complexity
print(f"Speedup ratio: {speedup_ratio:.2f} times faster than naive computation.")


preprocess time: 0.0
query dependent preprocess + candidates screening: 0.0
candidate ranking: 0.0
Top 10 candidates (indices):
[316 258 239 217 924 310  86 336 201 479]
Query-dependent runtime (seconds): 0.0000
Candidate ranking runtime (seconds): 0.0000
Total complexity: 3430
Top 10 candidates using naive approach (indices): [316 258 239 217 924 310  86  21 546 336]
Accuracy compared to naive method: 80.00%
Speedup ratio: 53.72 times faster than naive computation.


Netflix data

In [15]:
# Load preprocessed factors and biases
movie_factors = np.load("data/netflix/Movie_factors_15_new.npy")
movie_biases = np.load("data/netflix/Movie_biases_15_new.npy")
customer_factors = np.load("data/netflix/Customer_factors_15_new.npy")
customer_biases = np.load("data/netflix/Customer_biases_15_new.npy")
global_mean = np.load("data/netflix/netflix_global_mean.npy")

# Use movie factors as atoms for Greedy MIPS
atoms = movie_factors

# Choose a specific user
user_id = 0  # Replace with the desired user ID (index-based, starting at 0)
user_factors = customer_factors[user_id]
user_bias = customer_biases[user_id]

# Construct the user preference signal
signal = user_factors  # Optionally, add user_bias and global_mean if needed for personalization
signals = signal[np.newaxis, :]  # Ensure signal has 2D shape for batch processing

# Initialize Greedy MIPS parameters
budget = 10  # Number of candidates to screen
num_best_atoms = 10  # Top-k items to retrieve
conditer = generate_conditer(atoms)  # Query-independent preprocessing

# Run Greedy MIPS
best_atoms, query_times, ranking_times, complexities = greedy_mips(
    atoms=atoms,
    signals=signals,
    budget=budget,
    num_best_atoms=num_best_atoms,
    conditer=conditer,
    verbose=True
)

# Output results
print(f"Top {num_best_atoms} candidates (indices):\n{best_atoms[0]}")
print(f"Query-dependent runtime (seconds): {query_times[0]:.4f}")
print(f"Candidate ranking runtime (seconds): {ranking_times[0]:.4f}")
print(f"Total complexity: {complexities[0]}")

# Naive approach for validation
inner_products = np.dot(atoms, signal)
top_k_naive = np.argsort(inner_products)[-num_best_atoms:][::-1]
print(f"Top {num_best_atoms} candidates using naive approach (indices): {top_k_naive}")

# Compare results
accuracy = len(np.intersect1d(best_atoms[0], top_k_naive)) / num_best_atoms
print(f"Accuracy compared to naive method: {accuracy * 100:.2f}%")

# Compute speedup ratio
total_naive_computations = atoms.shape[0] * atoms.shape[1]
average_complexity = complexities.mean()
speedup_ratio = total_naive_computations / average_complexity
print(f"Speedup ratio: {speedup_ratio:.2f} times faster than naive computation.")


preprocess time: 0.0
query dependent preprocess + candidates screening: 0.0
candidate ranking: 0.0013377666473388672
Top 10 candidates (indices):
[ 130  105 1151  668  509  909  218  506 1131 1014]
Query-dependent runtime (seconds): 0.0000
Candidate ranking runtime (seconds): 0.0013
Total complexity: 7800
Top 10 candidates using naive approach (indices): [1274  738   46  559  314 1171 1305  305   24  217]
Accuracy compared to naive method: 0.00%
Speedup ratio: 17.31 times faster than naive computation.


Crypto pairs data

In [16]:
# Load the preprocessed dataset
dataset_path = "data/crypto_pairs/crypto_pairs_1m_dimensions.npy"  # Path to the saved .npy file
crypto_data = np.load(dataset_path, allow_pickle=True)

# Step 1: Use Truncated SVD to reduce dimensionality (if necessary)
svd = TruncatedSVD(n_components=50)  # Reduce to 50 dimensions
atoms = svd.fit_transform(crypto_data)  # The reduced dataset

# Step 2: Create a query signal
# Example: Use the first crypto pair as the query vector
query_index = 0
signal = atoms[query_index]  # A single crypto pair's embedding
signals = signal[np.newaxis, :]  # Ensure signal has 2D shape for batch processing

# Initialize Greedy MIPS parameters
budget = 10  # Number of candidates to screen
num_best_atoms = 10  # Top-k items to retrieve
conditer = generate_conditer(atoms)  # Query-independent preprocessing

# Run Greedy MIPS
best_atoms, query_times, ranking_times, complexities = greedy_mips(
    atoms=atoms,
    signals=signals,
    budget=budget,
    num_best_atoms=num_best_atoms,
    conditer=conditer,
    verbose=True
)

# Output results
print(f"Top {num_best_atoms} candidates (indices):\n{best_atoms[0]}")
print(f"Query-dependent runtime (seconds): {query_times[0]:.4f}")
print(f"Candidate ranking runtime (seconds): {ranking_times[0]:.4f}")
print(f"Total complexity: {complexities[0]}")

# Naive approach for validation
inner_products = np.dot(atoms, signal)
top_k_naive = np.argsort(inner_products)[-num_best_atoms:][::-1]
print(f"Top {num_best_atoms} candidates using naive approach (indices): {top_k_naive}")

# Compare results
accuracy = len(np.intersect1d(best_atoms[0], top_k_naive)) / num_best_atoms
print(f"Accuracy compared to naive method: {accuracy * 100:.2f}%")

# Compute speedup ratio
total_naive_computations = atoms.shape[0] * atoms.shape[1]
average_complexity = complexities.mean()
speedup_ratio = total_naive_computations / average_complexity
print(f"Speedup ratio: {speedup_ratio:.2f} times faster than naive computation.")


preprocess time: 0.0
query dependent preprocess + candidates screening: 0.031462907791137695
candidate ranking: 0.0006921291351318359
Top 10 candidates (indices):
[ 13  28  51   2  58  97  35 104  77  67]
Query-dependent runtime (seconds): 0.0315
Candidate ranking runtime (seconds): 0.0007
Total complexity: 3473
Top 10 candidates using naive approach (indices): [ 13  28  51   2  58  97  35 104  57  49]
Accuracy compared to naive method: 80.00%
Speedup ratio: 1.51 times faster than naive computation.
